In [1]:
import torch
import time

times = 5
# 14336 4096
h1 = 16384
h2 = 6144
dtype=torch.bfloat16
device1 = "cuda:1"
device2 = "cuda:2"
device3 = "cuda:3"

times_list = []
expert1 = torch.randn(h1, h2, dtype=dtype, device=device1)

time_start = time.time()
for i in range(times):
    time_start_once = time.time()
    out = expert1.to(device2)
    torch.cuda.synchronize(device=device1)
    torch.cuda.synchronize(device=device2)
    time_end_once = time.time()
    times_list.append(round(time_end_once - time_start_once, 6))
time_end = time.time()
print(f"Time taken: {time_end - time_start} seconds"
    f"Every time {times_list}"
)


/mnt/zhengcf3/env/lmp/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Time taken: 0.17659449577331543 secondsEvery time [0.105594, 0.017791, 0.017636, 0.017633, 0.017664]


In [2]:
import torch
import time

times = 5
h1 = 16384
h2 = 6144
dtype=torch.bfloat16
device1 = "cuda:1"
device2 = "cuda:2"
device3 = "cuda:3"


expert1 = torch.randn(h1, h2, dtype=dtype, device="cpu", pin_memory=True)

times_list = []
time_start = time.time()
for i in range(times):
    time_start_once = time.time()
    expert1.to(device2)
    time_end_once = time.time()
    times_list.append(round(time_end_once - time_start_once, 6))
time_end = time.time()
print(f"Time taken: {time_end - time_start} seconds"
    f"Every time {times_list}"
)


Time taken: 0.08314394950866699 secondsEvery time [0.016779, 0.016552, 0.016543, 0.016538, 0.016537]


In [2]:
import torch
import time
from transformers import MixtralForCausalLM, AutoModelForCausalLM


# 加载 Mixtral 模型（只加载一个 expert）
#Mixtral-8x22B-Instruct-v0.1
model_path = "/mnt/zhengcf3/models/Mixtral-8x7B"
print("正在加载模型...")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="cpu",  # 加载到 CPU
)


/mnt/zhengcf3/env/lmp/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/mnt/zhengcf3/env/lmp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


正在加载模型...


Loading checkpoint shards: 100%|██████████| 19/19 [00:02<00:00,  9.28it/s]


In [46]:

model.eval()
times = 5
h1 = 14336
h2 = 4096
dtype=torch.bfloat16
device1 = "cuda:1"
device2 = "cuda:2"
device3 = "cuda:3"

layer1 = model.model.layers[1]
layer2 = model.model.layers[2]
layer1 = layer1.to(device1)
layer2 = layer2.to(device2)

layer3 = model.model.layers[3]

layer1.eval()
layer2.eval()
batch_size = 64
seq_len = 8

inputs_cpu = torch.randn(batch_size, seq_len, h2, dtype=dtype, device="cpu")
inputsb0 = torch.randn(batch_size, seq_len, h2, dtype=dtype, device=device1)
inputsb1 = torch.randn(batch_size, seq_len, h2, dtype=dtype, device=device1)
inputsb2 = torch.randn(batch_size, seq_len, h2, dtype=dtype, device=device1)


inputs_list = [inputsb0]

w1 = torch.nn.Linear(h2, h1, bias=False, device=device1, dtype=dtype)
w2 = torch.nn.Linear(h1, h2, bias=False, device=device1, dtype=dtype)
w3 = torch.nn.Linear(h2, h1, bias=False, device=device1, dtype=dtype)
act_fn = torch.nn.SiLU()

def experts_cal(layer, inputs):
    for i in range(1):
        expert = layer.block_sparse_moe.experts[i]
        # for expert in layer.block_sparse_moe.experts:
        out = expert(inputs)
        print(out.shape)
    return out
def layer_cal(layer, inputs):
    bmoe = layer.block_sparse_moe
    out, _ = bmoe(inputs)
    return out

def move(tensor, device):
    return tensor.to(device)

times_list = []
time_start = time.time()
for i in range(times):
    time_start_once = time.time()
    for input in inputs_list:
        print(f"提交任务到worker1")
        out = experts_cal(layer3, inputs_cpu)
        torch.cuda.synchronize(device=device1)
        
    time_end_once = time.time()
    del out
    times_list.append(round(time_end_once - time_start_once, 6))
time_end = time.time()
print(f"Time taken: {time_end - time_start} seconds"
    f"Every time {times_list}"
)


提交任务到worker1
torch.Size([64, 8, 4096])
提交任务到worker1
torch.Size([64, 8, 4096])
提交任务到worker1
torch.Size([64, 8, 4096])
提交任务到worker1
torch.Size([64, 8, 4096])
提交任务到worker1
torch.Size([64, 8, 4096])
Time taken: 0.11647605895996094 secondsEvery time [0.031299, 0.021477, 0.020552, 0.020748, 0.021886]


In [ ]:
import time, torch
k = torch.randn(1408, 2048, dtype=torch.bfloat16, device="cpu", pin_memory=True)
time_start_torch = time.time()
for i in range(3):
    k_g = k.to("cuda:1")
time_end_torch = time.time()

print(f"allocate torch tensor time: {time_end_torch - time_start_torch} seconds")

allocate torch tensor time: 0.0009953975677490234 seconds
